In [4]:
# packages for today
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
import requests
from bs4 import BeautifulSoup
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from matplotlib.backends.backend_pdf import PdfPages
import seaborn as sns
import plotly.express as px
from sklearn.linear_model import Perceptron
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score
import statsmodels.api as sm
import math
from scipy.stats import norm 
import scipy.stats as stats
import pylab as py
from sklearn.preprocessing import PolynomialFeatures

### Read in Data

In [2]:
dogs_df = pd.read_csv('dogs.csv')
dogs_df.drop(columns='Unnamed: 0', inplace=True)
dogs_df.head()

,id,organization_id,url,age,gender,size,coat,tags,name,organization_animal_id,...,compatible_dogs,compatible_cats,spayed_neutered,house_trained,special_needs,shots_current,primary_color,secondary_color,tertiary_color,delta_time_minutes
0,59313263,CA2713,https://www.petfinder.com/dog/billy-59313263/c...,Adult,Male,Small,Short,"['Friendly', 'Affectionate', 'Funny', 'Couch p...",BILLY,NaN,...,True,True,True,True,False,True,Yellow / Tan / Blond / Fawn,NaN,NaN,1213.833333
1,59313260,IL54,https://www.petfinder.com/dog/ula-59313260/il/...,Adult,Female,Medium,Short,[],Ula,NaN,...,NaN,NaN,True,False,False,True,NaN,NaN,NaN,1015.283333
2,59313253,FL532,https://www.petfinder.com/dog/jake-59313253/fl...,Young,Male,Medium,Medium,"['Brave', 'Athletic', 'Funny', 'Smart', 'Loyal']",Jake,NaN,...,True,True,True,True,False,True,Merle (Blue),NaN,NaN,733.033333
3,59313248,CA2813,https://www.petfinder.com/dog/marshmallow-5931...,Young,Male,Large,Short,[],Marshmallow,NaN,...,True,NaN,True,True,True,True,White / Cream,NaN,NaN,1255.850000
4,59313245,TX258,https://www.petfinder.com/dog/riley-loves-ever...,Young,Female,Large,Short,"['Smart', 'Playful', 'Gentle', 'Loyal', 'Affec...",Riley- Loves Everyone and Learns Quickly!,NaN,...,True,NaN,True,True,False,True,Brindle,NaN,NaN,1359.433333


In [3]:
# drop null values for compatibility columns
dogs_df.dropna(subset=['compatible_dogs', 'compatible_cats', 'compatible_children', 'primary_color'], inplace=True)
dogs_df.reset_index(inplace=True)
dogs_df.head()

,index,id,organization_id,url,age,gender,size,coat,tags,name,...,compatible_dogs,compatible_cats,spayed_neutered,house_trained,special_needs,shots_current,primary_color,secondary_color,tertiary_color,delta_time_minutes
0,0,59313263,CA2713,https://www.petfinder.com/dog/billy-59313263/c...,Adult,Male,Small,Short,"['Friendly', 'Affectionate', 'Funny', 'Couch p...",BILLY,...,True,True,True,True,False,True,Yellow / Tan / Blond / Fawn,NaN,NaN,1213.833333
1,2,59313253,FL532,https://www.petfinder.com/dog/jake-59313253/fl...,Young,Male,Medium,Medium,"['Brave', 'Athletic', 'Funny', 'Smart', 'Loyal']",Jake,...,True,True,True,True,False,True,Merle (Blue),NaN,NaN,733.033333
2,8,59313185,AL444,https://www.petfinder.com/dog/springs-59313185...,Young,Female,Large,Short,"['Friendly', 'Affectionate', 'Quiet']",Springs,...,True,True,True,True,False,True,Bicolor,NaN,NaN,1269.450000
3,10,59263354,TX2594,https://www.petfinder.com/dog/ariel-59263354/t...,Baby,Female,Medium,Short,"['Friendly', 'Affectionate', 'Gentle', 'Playfu...",Ariel,...,True,True,False,False,False,True,Black,NaN,NaN,1127.616667
4,11,59313109,SC178,https://www.petfinder.com/dog/rocky-59313109/s...,Young,Male,Medium,Short,"['Friendly', 'Gentle', 'Loyal', 'Playful', 'Af...",Rocky,...,True,True,False,True,False,False,"Tricolor (Brown, Black, & White)",NaN,NaN,119.766667


### Multivariate Polynomial Regression 
This is our trial of a multivariate polynomial regression predicting delta time minutes from polynomial data for 'compatible_children', 'compatible_cats', and 'compatible_dogs'. Because our X data is made up of boolean columns, we later decided that a polynomial regression is not appropriate. However, we still would like to submit this code as it was part of the process of finding the final model. 

In [5]:
# build X and y data 
compat_kidsX = dogs_df['compatible_children'].to_numpy()
compat_catsX = dogs_df['compatible_cats'].to_numpy()
compat_dogsX = dogs_df['compatible_dogs'].to_numpy()

y = dogs_df['delta_time_minutes'].to_numpy()  
X = np.hstack((compat_kidsX.reshape(-1,1), compat_catsX.reshape(-1,1), compat_dogsX.reshape(-1, 1)))

In [6]:
from sklearn.preprocessing import PolynomialFeatures
from sklearn.metrics import mean_squared_error, r2_score

# run multivariate polynomial regression 
# change X data to be polynomial 
degree = 4
poly_feat = PolynomialFeatures(degree=degree)
X_poly = poly_feat.fit_transform(X)


# PolynomialFeatures will create a new matrix consisting of all polynomial combinations 
# of the features with a degree less than or equal to the degree we just gave the model (2)
poly_model = PolynomialFeatures(degree=degree)

# transform out polynomial features
X_poly = poly_model.fit_transform(X)

# let's fit the model
poly_model.fit(X_poly, y)

# we use linear regression as a base!!! ** sometimes misunderstood **
regression_model = LinearRegression()

regression_model.fit(X_poly, y)

y_pred = regression_model.predict(X_poly)

regression_model.coef_

mse = mean_squared_error(y, y_pred, squared=False)
r2 = r2_score(y, y_pred)

print(f'''The MSE is {mse} and the R2 is {r2}''')

The MSE is 494.67883298959725 and the R2 is 0.0015159001060781607
